<a href="https://colab.research.google.com/github/sokhan1/PYTHON/blob/main/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# capture는 주렁주렁 출력 보여주지 않게끔
# This cells is needed for displaying 'Korean character' clear
# *You have to run the cell once at the beginning and restart the runtime
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [3]:
%%capture
!pip install gdown # Enable user download 'sentence dataset'
!pip install transformers # I can use pre-trained model by b hugging
!pip install sentencepiece # This is needed for calling MarianTokenizer
!pip install sacremoses # Prevent 'warning' when importing from MarianMTModel
!pip install einops # (Einstein operations)

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd # To use excel file
from tqdm import tqdm
import math # To calculate PPL which is used to evaluate performance of 'NLP' model
from einops import rearrange

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cpu


## Pretrained model Sample

In [3]:
# Load the tokenizer & input embedding layer & last fc layer
tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ko-en') #korean-English Trans
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-ko-en') # MT: Machine Translation
# input_embedding = model.get_input_embeddings() # 일부 사이즈 큰 놈들은 데려와서 나머지만 학습시키려는 시도
# input_embedding.weight.requires_grad = False # freeze
# fc_out = model.get_output_embeddings()
# fc_out.weight.requires_grad = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/842k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [4]:
eos_idx = tokenizer.eos_token_id
pad_idx = tokenizer.pad_token_id
print("eos_idx=", eos_idx) #first token
print("pad_idx=", pad_idx) #last token

eos_idx= 0
pad_idx= 65000


##Hyper-parameter Setting

In [ ]:
BATCH_SIZE = 64 # In paper of 'Transformer', there are 25000 tokens in one batch
LAMBDA = 0 # For 12-Regularization
EPOCH = 15
max_len=100 # I think one sentence has at most 100 words(tokens), that's why I selected 100
criterion = nn.CrosEntropyLoss(ignore_index = pad_idx)

scheduler_name='Noam'

